In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sns.set_style('darkgrid')

In [2]:
df = pd.read_csv("")
df.shape

NameError: name 'pd' is not defined

In [ ]:
df.head()

In [ ]:
df['Vehicle_driver_relation'].value_counts()

In [ ]:
df.dtypes

In [ ]:
df.dtypes.value_counts()

In [ ]:
i = df[(df['Vehicle_driver_relation'] == 'Unknown')].index

In [ ]:
i = df[(df['Vehicle_driver_relation'] == 'Unknown')].index
df = df.drop(i)

In [ ]:
df.drop(['Sex_of_casualty'], inplace=True, axis=1)

In [ ]:
df.info()

In [ ]:
sns.countplot(data= df, x='Accident_severity')

In [ ]:
df.corr()

In [ ]:
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
t = '17:02:00'
t[:2]

In [ ]:
df['Day_of_week'].value_counts()

In [ ]:
def weekday_or_weekend(x):
    if x in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
        x = 'Weekday'
    else:
        x = 'Weekend'
    
    return x

In [ ]:
df['Day_type'] = df['Day_of_week'].apply(lambda x: weekday_or_weekend(x))
sns.countplot(data=df, x='Day_type')

In [ ]:
df.drop(['Day_of_week'], inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
def day_or_night(x):
    x = x[:2]
    if ":" in x:
        x = x[:1]
    
    x = int(x)
    
    if x >= 6 and x < 18:
        x = "Day"
    else:
        x = "Night"
    
    return x
    

In [ ]:
df['Time_of_day'] = df['Time'].apply(lambda x: day_or_night(x))

In [ ]:
sns.countplot(data=df,x='Time_of_day')

In [ ]:
df.drop(['Time'], inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
df = pd.concat([df.drop('Accident_severity', axis=1), pd.get_dummies(df['Accident_severity'], drop_first=True)], axis=1)
df.head()

In [ ]:
sns.histplot(data=df, x='Number_of_vehicles_involved')

In [ ]:
sns.histplot(data=df, x='Number_of_casualties')

In [ ]:
df.describe().transpose()

In [ ]:
df.describe(include='object').transpose()

In [ ]:
for col in df.select_dtypes(include='object'):
    if df[col].nunique() <= 30:
        sns.countplot(data=df, y=col)
        plt.show()

In [ ]:
df.nunique()